# Extract CIHA data from DATASUS

Import libraries

In [1]:
import glob
import os
from simpledbf import Dbf5
import urllib.request
import pandas as pd

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


Select state and year

In [2]:
estados = ["AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI",
           "RJ", "RN", "RS", "RO", "RR","SC", "SP", "SE", "TO"]
estado = estados[0] # Change here
ano = "17"
files = []
for mes in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
    files.append('CIHA' + estado + ano + mes)

Download DBC file

In [3]:
for file in files:
    url = 'ftp://ftp.datasus.gov.br/dissemin/publicos/CIHA/201101_/Dados/' + file + '.dbc'
    urllib.request.urlretrieve(url, file + '.dbc')

Convert DBC to DBF

In [4]:
for file in files:
    os.system('dbf2dbc ' + file + '.dbc')
    os.system('del /f ' + file + '.dbc')

Convert DBF to CSV

In [5]:
for file in files:
    dbf = Dbf5(file + '.dbf', codec='utf-8')
    dbf.to_csv(file + '.csv')

Delete DBF files

In [6]:
for file in files:
    os.system('del /f ' + file + '.dbf')

Merge all csv files

In [7]:
all_files = glob.glob("*.csv")
df = pd.concat((pd.read_csv(f, header = 0, low_memory=False, index_col=0) for f in all_files))

Choose columns of interest and create CSV

In [8]:
df = df[["DIAG_PRINC", "MUNIC_MOV"]]
df = df[df['DIAG_PRINC'].notnull()]
path = "ciha_by_state/"
df.to_csv(path + "CIHA" + estado + ano + ".csv")

Delete month files 

In [9]:
for file in files:
    os.system('del /f ' + file + '.csv')
